In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import random
from selenium.webdriver.common.keys import Keys
import json
from opencc import OpenCC
import pandas as pd
from time import sleep
from selenium.webdriver.common.action_chains import ActionChains
import re
from tqdm import tqdm

In [2]:
tag = pd.read_excel('./merchant_top10000.xlsx')
brands_10000 = []

for i in range(len(tag)):
  brands_10000.append(tag['merchant_cleasing_tag'][i])

brands_10000

['momo購物網',
 '統一超商',
 '優食台灣股份有限公司',
 'foodpanda',
 '樂購蝦皮',
 '中油',
 '全家便利商店',
 '富邦人壽',
 '全聯app',
 '台灣大哥大',
 '全聯福利中心',
 '電話費',
 '家樂福',
 '麥當勞',
 '悠遊卡自動加值',
 '優步',
 '高鐵',
 '富邦產物',
 '新光三越',
 '萊爾富',
 '寶雅',
 '遠東百貨',
 '50嵐',
 '屈臣氏',
 '星巴克',
 'linetaxi',
 '遠傳電信',
 '橘子支付',
 '台灣大車隊',
 '大潤發',
 '全盈支付金融科技股份有限公司',
 'line禮物',
 '美廉社',
 '康是美',
 '全國加油站',
 '可不可熟成紅茶',
 '台灣之星',
 'ikea宜家家居',
 '城市車旅',
 '八方雲集',
 'louisa coffee',
 '清心福全',
 'pchome',
 '台茂購物中心',
 '大樹連鎖藥局',
 '臺灣鐵路管理局網路購票',
 '迷客夏',
 '遠東sogo百貨',
 '誠品',
 '愛買',
 '東森得易購',
 '和雲行動服務',
 '蝦皮支付',
 'qburger',
 '麻古茶坊',
 '一卡通自動加值',
 '嘟嘟房',
 'globalmall',
 '爭鮮迴轉壽司',
 'uniqlo',
 '電費',
 '東森購物',
 'mitsui outlet',
 '遠東巨城購物中心',
 '摩斯漢堡',
 '必勝客',
 '雅虎奇摩購物中心',
 'worldgym',
 '亞太電信',
 '統一時代百貨',
 'yoxi',
 '麥味登',
 '肯德基',
 'coco',
 '小北百貨',
 '新東陽',
 '漢神巨蛋購物廣場',
 'decathlon迪卡儂',
 'global mall',
 '台灣自來水',
 'ok line pay',
 '統一精工',
 '北市停車費',
 '停車大聲公',
 '網購-91app',
 '京站時尚廣場',
 '美麗華百樂園',
 '電信資費',
 '85度c',
 'kfc',
 't startel',
 'goshare',
 'gogoro network',
 '交通部臺灣鐵路管理局',
 '義美股份有限

In [26]:
#取要爬的區間
#改的特店名：夢時代?統一時代 =>夢時代統一時代
#連加* =>連加*
# '拿坡里披薩?炸雞' => '拿坡里披薩炸雞'
# '星圓通訊-mo/rec' => '星圓通訊-mo'
brands1 = brands_10000[:50]
brands2 = brands_10000[50:100]
brands3 = brands_10000[100:112]
brands4  = brands_10000[112:200]
brands5  = brands_10000[200:289]
brands6 = brands_10000[289:300]
brands7 = brands_10000[300:400]
brands8 = brands_10000[400:425]
brands9 = brands_10000[425:437]
brands10 = brands_10000[437:448]
brands = brands_10000[448:500]

In [27]:
brands[0] = '星圓通訊-mo'

In [28]:

cc = OpenCC('s2tw')

#找到本瀏覽器的user agent
url = 'https://httpbin.org/headers'
browser = webdriver.Chrome('./chromedriver_win32/chromedriver')
browser.get(url)
user_agent = browser.page_source
browser.close()
try:
    match = re.search(r"\"User-Agent\":\s+\"(.*?)\",", user_agent).group(1)
except:
    match = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36'
print(match)


#記得創一個content的資料夾
fp_output = r'./content/result_{}.json'

#所有要爬的特店像這樣放在這個list中
#-------------------------------------------#
# brands = ['八方雲集', '我大巴', '舒服雷']
#-------------------------------------------#
#跟在特店名後面一起搜尋
keyword = '品牌'

#每個特店要爬幾頁
pagesss = 4

forloop_count = 0
for emo in tqdm(range(len(brands))):
    data_dic = {}
    page_num_dic = {}
    print(f'第{emo+1}間特店：{brands[emo]}，總共有{len(brands)}家')
    with open(f'./process.txt', "a+") as f:
        f.write(f'第{emo+1}間特店：{brands[emo]}，總共有{len(brands)}家\n')
    forloop_count+=1
    if forloop_count%10==0: #每10家休息200秒
        time.sleep(20) 
    time.sleep(10) #每家休息幾秒
    try:
        sleep(1)
        #options.add_argument("headless")
        options = webdriver.ChromeOptions()
        options.add_experimental_option('excludeSwitches', ['enable-logging'])
        options.add_argument(
            f'user-agent={match}')
        options.add_argument('--disable-blink-features=AutomationControlled')
        driver = webdriver.Chrome('./chromedriver_win32/chromedriver',options=options)
        driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
            "source": """
                Object.defineProperty(navigator, 'webdriver', {
                get: () => undefined
                })
            """
            })
        
        
        driver.implicitly_wait(5)
        driver.get('https://www.google.com') #navigating to google
        search_gog = driver.find_element(By.CLASS_NAME, "gLFyf")
        search_gog.send_keys(f'{brands[emo]} {keyword}')
        search_gog.send_keys(Keys.RETURN)
        li = []
        for i in range(pagesss):  # page number
            for des in driver.find_elements(By.CLASS_NAME, "VwiC3b.yXK7lf.MUxGbd.yDYNvb.lyLwlc.lEBKkf"):
                text = des.text
                li.append(cc.convert(text))
            try:
                next_links = driver.find_elements(By.LINK_TEXT, '下一頁')# note it is find_elements with an s
                next_links[0].location_once_scrolled_into_view
                actions = ActionChains(driver)
                time.sleep(random.uniform(3,6))
                # 滑鼠先移到 submitBtn 上，然後再點擊 submitBtn
                actions.move_to_element(next_links[0]).click(next_links[0])
                actions.perform()
            except:
                break

        data_dic[f'{brands[emo]}'] = li
        print(data_dic[f'{brands[emo]}'])
        page_num_dic[f'{brands[emo]}'] = i
        
        with open(fp_output.format(brands[emo]), 'w') as f:
             f.write(json.dumps(data_dic))
    finally:
#         input('pausing (hit enter to terminate) ...')
        driver.quit()

<ipython-input-28-2dc80a3ef207>:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome('./chromedriver_win32/chromedriver')


Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36


  0%|          | 0/52 [00:00<?, ?it/s]

第1間特店：星圓通訊-mo，總共有52家


<ipython-input-28-2dc80a3ef207>:48: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver_win32/chromedriver',options=options)


...

 96%|█████████▌| 50/52 [40:36<01:45, 52.92s/it]

第51間特店：股份有限公司，總共有52家
['公司介紹 ; 地址. 臺北市信義區忠孝東路四段512號5樓之2 ; 產業類別. 廣告行銷公關業 ; 產業描述. 品牌策略/視覺設計/包裝設計/網站設計/數位整合 ; 資本額. 2900萬元 經濟部 ...', '【公司簡介】品牌聯網於2010年投資設立宅妝服務，提供宅妝設計及售屋佈置服務，獲2014 ICT Best Aw...。公司位於新北市三重區。產業：專門設計相關業。', '元滋品牌股份有限公司，設立於: 104 年01 月，公司總部位於高雄市的前鎮區營業項目為餐飲品牌規劃與設計、連鎖餐飲業、原物料進出口、無店面零售業等，旗下相關品牌 ...', '中華店門市同仁、儲備幹部｜想找誠和品牌行銷股份有限公司更多的工作職缺，請上1111人力銀行搜尋：誠和品牌行銷股份有限公司,誠和品牌行銷股份有限公司職缺,誠和品牌 ...', '如果你需要的不只是一家設計公司， 而是陪你度過每個重要時刻，一起走向豐收之路的夥伴，歡迎來坐坐聊聊~禾望品牌股份有限公司隨時等著您的到來。', '工作經歷 ; 執行長. 配配品牌股份有限公司. 2021年1月 ; Founder. 安登股份有限公司. 2019年6月 ; R&D Technical Consultant. SiDC 衣騰科技. 2010年1月 ; Cofounder. 艾聚普 ...', '滿心企業股份有限公司，於1977年設立，1984年6月正式成立滿心企業股份有限公司。主要經營各種成衣針織品製造買賣，國內外知名服飾代理業務。經營方針為「求新、求精、 ...', '為品牌提供專屬配方,是我們服務客戶的理念。 我們是一個充滿活力的event house,包含大型發表會、巡迴活動、消費者活動、新品發表會、貴賓鑑賞會、展覽會場、賣場行銷.', '上汽名爵（英語：Morris Garages），是中國上海汽車集團股份有限公司的附屬品牌；曾為英國的汽車品牌，惟英國工廠於2016年停產，目前已成為中國自主國產品牌。', '在這裡，您可以找到有關成霖企業品牌、新產品和品牌相關內容的最新消息。', '產品一覽. 代理品牌 (102) ... 銷售品牌. 找不到符合您選擇的商品. 天隸企業股份有限公司. 臺北總公司：24158 新北市三重區中興北街131之18號; 電話： 02-2995-5800

 98%|█████████▊| 51/52 [41:25<00:51, 51.50s/it]

第52間特店：台北，總共有52家
['你所未見的臺北品牌識別(以下簡稱【臺北商標】)經過設計系統的延伸，創造出了品牌識別延伸插畫設計，提供了更多元的臺北觀光品牌元素應用。', '你所未見的臺北品牌識別(以下簡稱【臺北商標】)經過設計系統的延伸,創造出了品牌識別延伸插畫設計,. 提供了更多元的臺北觀光品牌元素應用。 聯名|持續擴大市場捲動產業.', '2021年3月11日 — 1.服飾品牌【lowrysfarm】 · 2.服飾品牌【heather】 · 3.服飾品牌【Jeanasis】 · 4.服飾品牌【Airspace】 · 5.服飾品牌【Queenshop】 · 6.服飾品牌【Corban】.', '2023年4月3日 — 今年的限定選品店首度進駐臺北SOGO復興館，集結超過40個臺灣設計師品牌，從衣服、包包到飾品配件，所有參與時裝週的品牌們，通通買得到！ 廣告- 內文未完 ...', "NIKE臺北品牌體驗店. ☎️02-27232860 MON-SUN 11:00AM-22:00PM. 8月新品's profile picture. 8月新品. 7月新品's profile picture. 7月新品.", '燒肉店設計 · 餐飲設計公司 · 和牛 · 森森logo · 燒肉設計. 森森燒肉Morimori Yakiniku. Save. 森森燒肉Morimori Yakiniku. Multiple Owners. 48 1.6k. 臺北的品牌 ...', '臺北信義新天地A11館位居臺灣二十一世紀超重量級信義商圈的樞紐，以職場新鮮人及雅痞家庭新族群為主力客層，為強調自我風格的新生活化購物商場；並且A11館與威秀影 ...', '從創業到品牌提供企業全方位獎勵補助，臺北市具有優質創業的軟硬實力，不論何種產業、領域，只要具有創意、特色及創新潛力，臺北市政府產業發展局透過產業發展獎勵補助的 ...', '2023年4月12日 — STIMLIG 臺北品牌店｜將生活拼湊成自己喜歡的模樣 ... 隱身在德惠街巷弄內的STIMLIG－臺北直營店，除了是STIMLIG 的北部展示體驗空間外，也是團隊的設計工作 ...', '2023年3月1日 — 瑞典品牌Acne Studios即將登臺插旗臺北信義區，. 近期越來越多品牌進駐臺灣啦！繼去年瑞典品牌COS 登臺之後，今日

100%|██████████| 52/52 [42:12<00:00, 48.70s/it]
